In [1]:
import pandas as pd
import warnings
import numpy as np
warnings.filterwarnings('ignore')



In [2]:
file_path='C:/Users/Administrator/Documents/kifiya/Week_10/data/preproccessed_brent_oil_prices.csv'
df = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True)
df = df.sort_values(by='Date')


# Clean Brent oil prices data
print("Column names in Brent oil prices data:", df.columns)
df.columns = df.columns.str.strip()
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Year'] = df['Date'].dt.year
df = df.dropna(subset=['Year'])
df['Year'] = df['Year'].astype(int)

display(df.head())


gdp_growth_path ='C:/Users/Administrator/Documents/kifiya/Week_10/data/gdp_growth.csv'
inflation_path = 'C:/Users/Administrator/Documents/kifiya/Week_10/data/inflation.csv'
exchange_rate_path = 'C:/Users/Administrator/Documents/kifiya/Week_10/data/exchangerates.csv'

Column names in Brent oil prices data: Index(['Date', 'Price', 'Returns', 'Log_Returns', 'Volatility', 'Momentum',
       'Rolling_Corr'],
      dtype='object')


,Date,Price,Returns,Log_Returns,Volatility,Momentum,Rolling_Corr,Year
0,1987-06-19,19.05,0.000000,0.000000,0.004308,0.37,NaN,1987
1,1987-06-22,19.10,0.002625,0.002621,0.003544,0.32,NaN,1987
2,1987-06-23,18.90,-0.010471,-0.010526,0.004313,0.00,NaN,1987
3,1987-06-24,18.75,-0.007937,-0.007968,0.004698,-0.28,NaN,1987
4,1987-06-25,18.70,-0.002667,-0.002670,0.004736,-0.35,NaN,1987


In [3]:
def preprocess_economic_data(file_path, country, data_type):
    # Load the dataset
    df1 = pd.read_csv(file_path, skiprows=4)

    # Filter for the specified country; consider adding a check for 'All' or specific country handling
    df1 = df1[df1['Country Name'] == country] if country != 'All' else df1

    # Drop unnecessary columns
    df1.drop(columns=['Country Code', 'Indicator Name', 'Indicator Code'], inplace=True)

    # Remove columns with names that are not numeric (e.g., "Unnamed: 68")
    df1 = df1.loc[:, ~df1.columns.str.contains('^Unnamed')]

    # Melt the dataframe to long format
    df1 = df1.melt(id_vars=['Country Name'], var_name='Year', value_name='Value')

    # Convert 'Year' to numeric, forcing errors to NaN, then drop NaNs
    df1['Year'] = pd.to_numeric(df1['Year'], errors='coerce')
    df1.dropna(subset=['Year'], inplace=True)

    # Pivot the DataFrame to have years as index
    df1 = df1.pivot(index='Year', columns='Country Name', values='Value')

    # Drop rows (years) with all NaN values
    df1.dropna(axis=0, how='all', inplace=True)

    return df1

# Preprocess each dataset for 'All' countries
country = 'All'
gdp_growth_clean = preprocess_economic_data(gdp_growth_path, country, 'GDP Growth Rates')
inflation_rates_clean = preprocess_economic_data(inflation_path, country, 'Inflation Rates')
exchange_rates_clean = preprocess_economic_data(exchange_rate_path, country, 'Exchange Rates')

# Merge datasets with the main DataFrame 'df'
merged_data = df.merge(gdp_growth_clean, on='Year', how='left', suffixes=('', '_GDP'))
merged_data = merged_data.merge(inflation_rates_clean, on='Year', how='left', suffixes=('', '_Inflation'))
merged_data = merged_data.merge(exchange_rates_clean, on='Year', how='left', suffixes=('', '_Exchange'))

# Display the merged data
print("Merged Data:")
display(merged_data.head())


Merged Data:


,Date,Price,Returns,Log_Returns,Volatility,Momentum,Rolling_Corr,Year,Afghanistan,Africa Eastern and Southern,...,Uzbekistan_Exchange,Vanuatu_Exchange,"Venezuela, RB_Exchange",Viet Nam_Exchange,Virgin Islands (U.S.)_Exchange,West Bank and Gaza_Exchange,World_Exchange,"Yemen, Rep._Exchange",Zambia_Exchange,Zimbabwe_Exchange
0,1987-06-19,19.05,0.000000,0.000000,0.004308,0.37,NaN,1987,NaN,3.964516,...,NaN,109.849167,0.0145,78.953316,NaN,NaN,NaN,NaN,0.009519,0.001663
1,1987-06-22,19.10,0.002625,0.002621,0.003544,0.32,NaN,1987,NaN,3.964516,...,NaN,109.849167,0.0145,78.953316,NaN,NaN,NaN,NaN,0.009519,0.001663
2,1987-06-23,18.90,-0.010471,-0.010526,0.004313,0.00,NaN,1987,NaN,3.964516,...,NaN,109.849167,0.0145,78.953316,NaN,NaN,NaN,NaN,0.009519,0.001663
3,1987-06-24,18.75,-0.007937,-0.007968,0.004698,-0.28,NaN,1987,NaN,3.964516,...,NaN,109.849167,0.0145,78.953316,NaN,NaN,NaN,NaN,0.009519,0.001663
4,1987-06-25,18.70,-0.002667,-0.002670,0.004736,-0.35,NaN,1987,NaN,3.964516,...,NaN,109.849167,0.0145,78.953316,NaN,NaN,NaN,NaN,0.009519,0.001663


In [4]:
#Drop columns that contain only NaN values
merged_data.dropna(axis=1, how='all', inplace=True)
# Drop non-numeric columns if there are any left after cleaning
merged_data = merged_data.select_dtypes(include=[np.number])

# Handle remaining missing values if needed
merged_data.fillna(method='ffill', inplace=True)  # Forward-fill remaining NaNs if needed
merged_data.dropna(inplace=True)  # Drop any rows with remaining NaNs
# Display the merged data
print("Merged Data:")
display(merged_data.head())


Merged Data:


,Price,Returns,Log_Returns,Volatility,Momentum,Rolling_Corr,Year,Afghanistan,Africa Eastern and Southern,Africa Western and Central,...,United Kingdom_Exchange,United States_Exchange,Uruguay_Exchange,Uzbekistan_Exchange,Vanuatu_Exchange,"Venezuela, RB_Exchange",Viet Nam_Exchange,"Yemen, Rep._Exchange",Zambia_Exchange,Zimbabwe_Exchange
8517,50.37,-0.016595,-0.016734,0.014191,-0.51,0.053800,2021,-20.738839,4.49738,3.994854,...,0.727065,1.0,43.554575,10609.464391,109.4525,9.975,23159.782593,1028.107754,20.018487,88.552447
8518,53.16,0.055390,0.053910,0.018218,2.28,0.139128,2021,-20.738839,4.49738,3.994854,...,0.727065,1.0,43.554575,10609.464391,109.4525,9.975,23159.782593,1028.107754,20.018487,88.552447
8519,53.80,0.012039,0.011967,0.018143,3.36,0.096925,2021,-20.738839,4.49738,3.994854,...,0.727065,1.0,43.554575,10609.464391,109.4525,9.975,23159.782593,1028.107754,20.018487,88.552447
8520,53.70,-0.001859,-0.001860,0.017920,2.96,0.242628,2021,-20.738839,4.49738,3.994854,...,0.727065,1.0,43.554575,10609.464391,109.4525,9.975,23159.782593,1028.107754,20.018487,88.552447
8521,55.51,0.033706,0.033150,0.018989,4.29,0.364805,2021,-20.738839,4.49738,3.994854,...,0.727065,1.0,43.554575,10609.464391,109.4525,9.975,23159.782593,1028.107754,20.018487,88.552447


In [5]:
merged_data.to_csv("C:/Users/Administrator/Documents/kifiya/Week_10/data/Merged_brent_oil_prices_with_Indicators.csv", index= True)
print("Merged file saved")
# The merged_data DataFrame is now clean and ready for scaling and modeling
print("Cleaned Data Summary:")
print(merged_data.info())

Merged file saved
Cleaned Data Summary:
<class 'pandas.core.frame.DataFrame'>
Index: 473 entries, 8517 to 8989
Columns: 711 entries, Price to Zimbabwe_Exchange
dtypes: float64(710), int32(1)
memory usage: 2.6 MB
None
